## Data Collection code 

#### This notebook provides code to reproduce the primary data used in data analysis and training the models.


In [3]:
import configparser
from TwitterAPI import TwitterAPI
import json
import csv
import sys
import time
import pandas as pd
import ast
import numpy as np
import collections
import os
import math
import collections
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA,TruncatedSVD
import re
import random
from emoji_function import demojize
import nltk
from nltk.corpus import stopwords
import json
from myUtility import get_twitter, filePrepration

In [26]:
def dataCollection(rootAddress, maxNumTweets):
    
    """
        The data collection function
        Params:
          rootAddress....A string representing the rood directory
          maxNumTweets...An intiger representing the number of tweets 
                        to be collected
       
    """
    twitter = get_twitter('twitter.cfg')
    print('Established Twitter connection.')
    
    filePrepration(rootAddress, ["dual_friendship", 
                             "no_friendship", 
                             "target_follow_sender_friendship", 
                             "sender_follow_target_friendship"])
    
    ## read the seedwords
    with open("seedWords.json", "r") as read_file:
        seedWords = json.load(read_file)
        seedWords = seedWords["hostile_words"]
    counter = 0
    
    ##
    req = twitter.request('statuses/filter', {'track': ' ,'.join(seedWords), 'lang':'en'})
    
    while True:
        try:
            for tweet in req.get_iterator():
                keys = tweet.keys()
                if 'in_reply_to_user_id' in keys:
                    ## filter the starter_tweets and the tweets with the same target and sender
                    if tweet['in_reply_to_user_id']!= None and tweet['user']['id']!= tweet['in_reply_to_user_id']:
                        target = tweet['in_reply_to_user_id']
                        source = tweet['user']['id']
                        request = twitter.request('friendships/show', {'source_id':source, 'target_id':target})
                        
                        ## check the relation between the sender and the target of tweet
                        for relation in request:
                            fileName = ''
                            relation = relation['relationship']['source']
                            if relation['following']:
                                if relation['followed_by']:
                                    fileName = "dual_friendship"
                                else:
                                    fileName = "sender_follow_target_friendship"
                            elif relation['followed_by']:
                                fileName = "target_follow_sender_friendship"
                            else:
                                fileName = "no_friendship"
                                
                            ## store the tweet in the file that presents the relationship type
                            with open(rootAddress + fileName + ".json", mode='a', encoding='utf-8') as feedsjson:
                                feedsjson.write(json.dumps(tweet)+"\n")
                            feedsjson.close()
                        counter += 1
                        print(counter)
                if counter > (maxNumTweets):
                    break
        ## handle the twitter rate limitation ( sleep for 15 mins )
        except Exception as e: 
            print("---------------")
            print(e)
            print("---------------")
            print('Got error %s \nsleeping for 15 minutes.' % request.text)
            print(request.status_code)
            sys.stderr.flush()
            time.sleep(61 * 15)
            twitter = get_twitter('twitter.cfg')
            req = twitter.request('statuses/filter', {'track': ' ,'.join(seedWords), 'lang':'en'})

In [ ]:
if __name__ == "__main__":
    rootAddress = "./"
    dataCollection(rootAddress, 10**6)